In [27]:
import pandas as pd
import numpy as np
train = pd.read_csv("data/announcements.csv")

In [13]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

# Download the punkt tokenizer for sentence splitting
import nltk.data
# nltk.download('punkt')   

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [17]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from default set")
for review in train["Body"]:
    sentences += review_to_sentences(str(review), tokenizer)

Parsing sentences from default set


/opt/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/opt/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:207: UserWarning: "b'http://www.bestbuy.com/site/Insignia%26%23153%3B+-+26%26%2334%3B+Class+-+LED+-+1080p+-+60Hz+-+HDTV/5975302.p;jsessionid=4174C8AE162899F3CAC0055456A6E39E.bbolsp-app02-13?id=1218716792876&skuId=5975302&st=insignia%20led%20tv&cp=1&lp=3'" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" loo

In [18]:
print(len(sentences))

16604


In [19]:
print (sentences[0])

['happy', 'new', 'year', 'everyone']


In [20]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 100    # Word vector dimensionality                      
min_word_count = 5   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "100features_5minwords_10context_AnnouncementTitle"
model.save(model_name)

Training model...


In [21]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = int(word_vectors.shape[0] / 5)

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  16.41153621673584 seconds.


In [22]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

In [34]:
num_clusters

764

In [35]:
# For the first 10 clusters
for cluster in range(0,50):
    #
    # Print the cluster number  
    print ("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    v=list(word_centroid_map.values())
    k = list(word_centroid_map.keys())
    for i in range(0,len(word_centroid_map.values())):
        if( v[i] == cluster ):
            words.append(k[i])
    print (words)


Cluster 0
['picture']

Cluster 1
['consists', 'religion', 'regularly', 'studio', 'country', 'determined', 'community', 'trials', 'anticipated', 'autonomous', 'gender', 'characteristics', 'cross', 'engineer', 'faster', 'art', 'explore', 'activity', 'success', 'apply']

Cluster 2
['externalclassd', 'baf', 'db', 'ae', 'ce', 'c', 'aac', 'bf', 'aa', 'externalclass', 'cd', 'b', 'ea', 'cf', 'cc', 'bd', 'dd', 'da', 'f', 'ca']

Cluster 3
['jan', 'plane', 'noon', 'afternoon', 'tonight', 'indi', 'nov', 'evening']

Cluster 4
['meadowla', 'klineal', 'advance', 'thompsm', 'kepchaja', 'munstebc']

Cluster 5
['sized', 'boy', 'graphing', 'fridge', 'drawer', 'ex', 'clip', 'originally', 'smartphone', 'sofa', 'jbl', 'enclosure', 'shelves']

Cluster 6
['no', 'decided', 'blank', 'only']

Cluster 7
['logic', 'solving', 'norton', 'source', 'sciences', 'meetings', 'youth', 'worlds', 'cyber', 'approach', 'ultimate', 'computational', 'theory', 'analysis', 'mass', 'finance', 'presents', 'managing', 'ghahramani']

In [25]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [28]:
#Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["Body"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( train["Body"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

NameError: name 'clean_train_reviews' is not defined